# This is the workbook for exact NO rotation that transforms the system into a solvable system

In [1]:
import numpy as np
from dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc
from dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
import dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from block2 import SZ as block2_SZ
from get_dmrg_config import get_dmrg_process_param, get_dmrg_config
from pathlib import Path
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from pyblock2._pyscf.ao2mo import integrals as itg
import pyscf
from OrbitalRotation.eigenstate_check import check_eigenstates
from basic_definitions import chemist_to_physicist, get_ferm_op, physicist_to_chemist
from Hamiltonians.molecular_hamiltonians import get_n2, get_h2o, get_hf, get_he, get_co, get_h2
import openfermion as of
from Hamiltonians.dmrg_hamiltonians import get_dmrg_hamiltonian, dmrg_non_exact

In [4]:
data = get_h2o(3.0)

ncas, n_elec, spin, h1e, g2e = data.ncas, data.n_elec, data.spin, data.h1e, data.g2e
print(ncas)

converged SCF energy = -74.2710321184149
7


In [2]:
from pyscf import gto, scf

mol = gto.M(
    atom=f"N 0 0 0; N 0 0 2.2",
    basis='sto-3g')
mf = scf.RHF(mol).run(conv_tol=1E-14)
mycc = mf.CISD().run()
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
                                                                     ncore=0,
                                                                     g2e_symm=1)

print(f"SCF energy: {mf.kernel() - ecore}")
print(f"CISD energy: {mycc.e_tot - ecore}")

converged SCF energy = -106.751831266188
E(RCISD) = -107.2227326317978  E_corr = -0.4709013656102005
converged SCF energy = -106.751831266188
SCF energy: -118.53805096395126
CISD energy: -119.00895232956147


In [3]:
obt, tbt = physicist_to_chemist(h1e.copy(), g2e.copy())
print(tbt)

[[[[ 1.09390225e+00  1.32639108e-06 -2.04165725e-07 ... -5.36844789e-17
    -4.60401003e-17  1.48362784e-02]
   [ 1.32639108e-06  1.09414258e+00 -9.99089267e-02 ... -1.17958490e-15
    -5.54718462e-16  1.34353844e-08]
   [-2.04165725e-07 -9.99089267e-02  2.99516310e-01 ... -6.87560611e-16
    -2.64737771e-16 -4.09301206e-08]
   ...
   [-5.36844789e-17 -1.17958490e-15 -6.87560611e-16 ...  3.02331744e-01
     4.18731981e-18 -6.44706950e-17]
   [-4.60401003e-17 -5.54718462e-16 -2.64737771e-16 ...  4.18731981e-18
     3.02331744e-01 -4.02560354e-17]
   [ 1.48362784e-02  1.34353844e-08 -4.09301206e-08 ... -6.44706950e-17
    -4.02560354e-17  3.17366920e-01]]

  [[ 1.32639108e-06  9.73870066e-01 -9.99252498e-02 ... -9.22977728e-16
    -4.58857285e-16  1.17881275e-08]
   [ 9.73870066e-01 -1.32606344e-06  6.79547367e-08 ... -1.39896505e-17
    -3.67859049e-17  1.72832657e-02]
   [-9.99252498e-02  6.79547367e-08  1.99227705e-10 ...  2.50051838e-16
    -5.93093062e-18 -3.00495358e-02]
   ...
   

In [ ]:
obt, tbt = physicist_to_chemist(h1e.copy(), g2e.copy())
ferm_op = get_ferm_op(obt, spin_orb = False)
ferm_op += get_ferm_op(tbt, spin_orb = False)
print(ferm_op)
sparse_H_tmp = of.get_sparse_operator(ferm_op)
ed_energy, t_sol = of.get_ground_state(sparse_H_tmp)

-27.99910600983559 [0^ 0] +
1.0939022464337156 [0^ 0 0^ 0] +
1.3263910820531415e-06 [0^ 0 0^ 2] +
-2.0416572533470738e-07 [0^ 0 0^ 4] +
0.10363690650847858 [0^ 0 0^ 6] +
-2.5488125448431316e-08 [0^ 0 0^ 8] +
0.014836278393073995 [0^ 0 0^ 18] +
1.0939022464337156 [0^ 0 1^ 1] +
1.3263910820531415e-06 [0^ 0 1^ 3] +
-2.0416572533470738e-07 [0^ 0 1^ 5] +
0.10363690650847858 [0^ 0 1^ 7] +
-2.5488125448431316e-08 [0^ 0 1^ 9] +
0.014836278393073995 [0^ 0 1^ 19] +
1.3263910820531415e-06 [0^ 0 2^ 0] +
1.0941425836669028 [0^ 0 2^ 2] +
-0.09990892671782572 [0^ 0 2^ 4] +
7.046283061005591e-08 [0^ 0 2^ 6] +
-0.013708071504523607 [0^ 0 2^ 8] +
1.343538438786589e-08 [0^ 0 2^ 18] +
1.3263910820531415e-06 [0^ 0 3^ 1] +
1.0941425836669028 [0^ 0 3^ 3] +
-0.09990892671782572 [0^ 0 3^ 5] +
7.046283061005591e-08 [0^ 0 3^ 7] +
-0.013708071504523607 [0^ 0 3^ 9] +
1.343538438786589e-08 [0^ 0 3^ 19] +
-2.0416572533470738e-07 [0^ 0 4^ 0] +
-0.09990892671782572 [0^ 0 4^ 2] +
0.29951631003289525 [0^ 0 4^ 4] +
-2.42

In [9]:
print(f"FCI energy: {ed_energy}")

FCI energy: -83.86029616561716


In [4]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)

### Given h1e, g2e, the following returns the physicist 
absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e.copy()), np.array(g2e.copy()), norb=num_orbitals, nelec=num_electrons, fac=0.5)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e.copy()).dtype, copy=False)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, h_in_tbt)

In [5]:
import shutil
import os

# Define the directory path
dir_path = "./mps_directory"
dir_path2 = "./mps_directory_2"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")
    
if os.path.exists(dir_path2):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path2)
    print(f"Deleted '{dir_path2}' and all its contents.")
else:
    print(f"Directory '{dir_path2}' does not exist.")

Deleted './mps_directory' and all its contents.
Deleted './mps_directory_2' and all its contents.


In [6]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =      17994

Build MPO | Nsites =    10 | Nterms =      17994 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.002 T = 0.013
 Site =     1 /    10 .. Mmpo =    82 DW = 0.00e+00 NNZ =      479 SPT = 0.8053 Tmvc = 0.004 T = 0.019
 Site =     2 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =      939 SPT = 0.9091 Tmvc = 0.004 T = 0.021
 Site =     3 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =     1579 SPT = 0.9326 Tmvc = 0.003 T = 0.018
 Site =     4 /    10 .. Mmpo =   262 DW = 0.00e+00 NNZ =     2203 SPT = 0.9548 Tmvc = 0.003 T = 0.014
 Site =     5 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =     8147 SPT = 0.8328 Tmvc = 0.002 T = 0.018
 Site =     6 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =     1575 SPT = 0.9328 Tmvc = 0.001 T = 0.006
 Site =     7 /    10 .. Mmpo =    82 DW = 0.00e+00 NNZ =      943 SPT = 0.9087 Tmvc = 0.000 T = 0.005
 Site =     8 /    10 .. Mmpo =

In [7]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")
mps_dir = Path("./mps_directory/mps_storage/dmrg_loop_035_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)
tools.init(block2_SZ)
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)
variance = check_eigenstates(num_orbitals, num_electrons, num_unpaired, obt, tbt, "dmrg_loop_035_ket_optimized", "mps_directory")
print(f"Energy variance: {variance}")

cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_

mps center changed (temporarily)
DTRIE T =      0.813
Number of DET =      14399 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 2222222000  =    0.392773851775139
DET          1 2222220020  =   -0.311907497800855
DET          2 2222202200  =   -0.311907497800512
DET          3 2222200220  =    0.277377350365759
DET          4 22222abab0  =    0.200550583210019
DET          5 22222baba0  =    0.200550583208726
DET          6 2222022002  =   -0.198757782608222
DET          7 2222020022  =    0.173325523825854
DET          8 2222002202  =    0.173325523825693
DET          9 2222000222  =   -0.169484458584456
DET         10 2222ba2a0b  =    0.144143722124976
DET         11 2222ab2b0a  =    0.144143722124781
DET         12 2222b2a0ab  =    0.144143722124629
DET         13 2222a2b0ba  =    0.144143722124499
DET         14 2222ba0a2b  =   -0.126777432196584
DET         15 2222ab0b2a  =   -0.126777432196316
DET         16 2222b0a2ab  =   -0.126777432

cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory/mps_storage/dmrg_loop_035_ket_optimized/F.MPS.INFO.ket_

Energy variance: 2.6973983046972182e-06


In [8]:
pdm = driver.get_1pdm(mps)

# One spin pdm either alpha or beta
rdm = pdm[0]

from scipy.linalg import eigh
import numpy as np
eigenvalues, eigenvectors = eigh(rdm)

# Verify eigenvectors being unitary
product = eigenvectors.conj().T @ eigenvectors
print(np.allclose(product, np.eye(eigenvectors.shape[0])))

U_NO = eigenvectors.conj().T

True


In [9]:
absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e), np.array(g2e), norb=num_orbitals, nelec=num_electrons)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e).dtype, copy=False)

In [10]:
p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt)[0]
orbital_rotated = np.einsum('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt, optimize = p)

In [11]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, 0.5 * orbital_rotated)

In [13]:
label = "_NO"
filename = "fcidump.N2_22" + f"{label}"
pyscf.tools.fcidump.from_integrals(
    "fcidump_output"/Path(filename),
    obt,
    tbt,
    nmo=obt.shape[0],
    nelec=n_elec,
    nuc=ecore,
    ms=spin,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [21]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory_2", verbosity=2, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       4676

Build MPO | Nsites =     7 | Nterms =       4676 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2237 SPT = 0.8279 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.013 Tmvc-total = 0.002 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4157 SIZE =        33216 SPT = 0.8748

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4676

Build MPO | Nsites =     7 | Nterms =       4676 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2237 SPT = 0.8279 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4157 SIZE =        33216 SPT = 0.8748

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4676

Build MPO | Nsites =     7 | Nterms =       4676 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2237 SPT = 0.8279 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4157 SIZE =        33216 SPT = 0.8748

...

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm
/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:345: RuntimeWarning: divide by zero encountered in log
  ln_del_E_rel = a + b * np.log(discarded_weights)


integral cutoff error =  0.0
mpo terms =       4676

Build MPO | Nsites =     7 | Nterms =       4676 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2237 SPT = 0.8279 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4157 SIZE =        33216 SPT = 0.8748

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm
/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:345: RuntimeWarning: divide by zero encountered in log
  ln_del_E_rel = a + b * np.log(discarded_weights)


integral cutoff error =  0.0
mpo terms =       4676

Build MPO | Nsites =     7 | Nterms =       4676 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2237 SPT = 0.8279 Tmvc = 0.000 T = 0.004
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.013 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4157 SIZE =        33216 SPT = 0.8748

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm
/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:345: RuntimeWarning: divide by zero encountered in log
  ln_del_E_rel = a + b * np.log(discarded_weights)


# Non-exact case

In [23]:
mol = gto.M(
        atom=f"O 0 0 0; H  0 1.0 0; H 0 0 1.0",
        basis='sto-3g')
mf = scf.RHF(mol).run(conv_tol=1E-14)
mycc = mf.CISD().run()
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
                                                                     ncore=0,
                                                                     g2e_symm=1)

### Given h1e, g2e, the following returns the physicist 
absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e.copy()), np.array(g2e.copy()), norb=num_orbitals, nelec=num_electrons, fac=0.5)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e.copy()).dtype, copy=False)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, h_in_tbt)

print(f"SCF energy: {mf.kernel() - ecore}")
print(f"CISD energy: {mycc.e_tot - ecore}")

converged SCF energy = -74.9611711378677
E(RCISD) = -75.01826559023741  E_corr = -0.0570944523697217
converged SCF energy = -74.9611711378677
SCF energy: -83.8021913068786
CISD energy: -83.85928575924832


In [34]:
energy_threshold = 1e-16

In [35]:

bd_estimated = dmrg_non_exact(mol, obt, tbt, ed_energy, energy_threshold)

converged SCF energy = -74.9611711378677
-83.86029616561716
converged SCF energy = -74.9611711378677
-83.42867492868548
-83.42441140305458
-83.42441140305459
-83.42441140305458
-83.42441140305458
-83.42441140305458
-83.42441140305453
-83.42441140305456
-83.42441140305458
converged SCF energy = -74.9611711378677
-83.86029616561706
-83.8585812285686
-83.85858122856864
-83.85858142363915
-83.85858142363902
-83.85859246243379
-83.85859246243339
-83.8585923443282
-83.85859234432867
converged SCF energy = -74.9611711378677
-83.86029616561781
-83.86013985251218
-83.860139852512
-83.86013985229636
-83.86013985229616
-83.86016427352865
-83.86016427352861
-83.86016427364734
-83.86016427364738
converged SCF energy = -74.9611711378677
-83.86029616561802
-83.86021555311262
-83.86021555311274
-83.86021555311075
-83.86025248779707
-83.86025423757147
-83.86025423757113
-83.86025423757799
-83.86025423757769
converged SCF energy = -74.9611711378677
-83.86029616561822
-83.8602715630366
-83.86027156904976

In [36]:
print(bd_estimated)

61


In [37]:
obt, tbt = chemist_to_physicist(one_body_dummy, 0.5 * orbital_rotated)

In [38]:
bd_estimated_2 = dmrg_non_exact(mol, obt, tbt, ed_energy, energy_threshold)

print(bd_estimated_2)

converged SCF energy = -74.9611711378677
-83.86029616561716
converged SCF energy = -74.9611711378677
-83.42867492868535
-83.42441140305459
-83.42441140305458
-83.42441140305459
-83.42441140305459
-83.4244114030546
-83.42441140305459
-83.42441140305459
-83.42441140305456
converged SCF energy = -74.9611711378677
-83.860296165616
-83.85858122131054
-83.85858122131064
-83.85858142389293
-83.85858142389331
-83.85859243910353
-83.85859243910335
-83.85859234258477
-83.85859234258507
converged SCF energy = -74.9611711378677
-83.86029616561773
-83.86013985251215
-83.86013985251186
-83.86013985229638
-83.86013985229613
-83.86016427352857
-83.86016427352845
-83.8601642736474
-83.86016427364721
converged SCF energy = -74.9611711378677
-83.8602961656171
-83.86021555311271
-83.86021555311268
-83.86021555311088
-83.86025248779792
-83.86025423757107
-83.86025423757121
-83.86025423757759
-83.86025423757772
converged SCF energy = -74.9611711378677
-83.8602961656163
-83.86027156303643
-83.8602715690483
-